In [65]:
import pandas as pd
import numpy as np
import networkx as nx
from typing import List

In [20]:
fname = "../data/huang_emt_scc.gxml"
# fname = "../data/omnipath.gxml"
g = nx.read_graphml(fname)
cycles = sorted(nx.simple_cycles(g))

# Positive-cycle
A cycle is positive if the XOR of their "represses" of all the edges results in a positive

In [63]:
cycle = cycles[0]
positive = True
for i, node in enumerate(cycle):
    next_node = cycle[(i + 1) % len(cycle)]
    repress = g.edges[node, next_node]['repress']
    positive ^= repress
    # print(f" -> ({node}, {next_node}: {repress}) -> {positive}", end="")
print(cycle)

['0', '1', '11', '10', '12', '13']


## Type 1 Motif

## Find all positive cycles

In [ ]:
def is_positive(graph: nx.Graph, cycle: List[str]):
    positive = True
    for i, node in enumerate(cycle):
        next_node = cycle[(i + 1) % len(cycle)]
        repress = g.edges[node, next_node]['repress']
        positive ^= repress
    return positive

In [59]:
mp = dict()
for i, c in enumerate(cycles):
    subg = g.subgraph(c)
    mp[subg] = i
print(mp)

{<networkx.classes.digraph.DiGraph object at 0x119177470>: 0, <networkx.classes.digraph.DiGraph object at 0x1191744a0>: 1, <networkx.classes.digraph.DiGraph object at 0x119174650>: 2, <networkx.classes.digraph.DiGraph object at 0x119177aa0>: 3, <networkx.classes.digraph.DiGraph object at 0x119177890>: 4, <networkx.classes.digraph.DiGraph object at 0x119176c00>: 5, <networkx.classes.digraph.DiGraph object at 0x119177020>: 6, <networkx.classes.digraph.DiGraph object at 0x119175610>: 7, <networkx.classes.digraph.DiGraph object at 0x1191778c0>: 8, <networkx.classes.digraph.DiGraph object at 0x119177d10>: 9, <networkx.classes.digraph.DiGraph object at 0x119177ef0>: 10, <networkx.classes.digraph.DiGraph object at 0x119177da0>: 11, <networkx.classes.digraph.DiGraph object at 0x119177ec0>: 12, <networkx.classes.digraph.DiGraph object at 0x119177e90>: 13, <networkx.classes.digraph.DiGraph object at 0x1191778f0>: 14, <networkx.classes.digraph.DiGraph object at 0x119106cc0>: 15, <networkx.classes

In [14]:
for i, n in enumerate(g.nodes()):
    g.nodes[n]["index"]  = (i *15) // 2
    print(g.nodes[n])

{'name': 'FOXC2', 'index': 0}
{'name': 'ZEB1', 'index': 7}
{'name': 'miR-101', 'index': 15}
{'name': 'ZEB2', 'index': 22}
{'name': 'SNAI1', 'index': 30}
{'name': 'miR-141', 'index': 37}
{'name': 'TGF-beta', 'index': 45}
{'name': 'miR-200a', 'index': 52}
{'name': 'miR-200b', 'index': 60}
{'name': 'miR-200c', 'index': 67}
{'name': 'SNAI2', 'index': 75}
{'name': 'miR-34a', 'index': 82}
{'name': 'TWIST2', 'index': 90}
{'name': 'TWIST1', 'index': 97}
{'name': 'GSC', 'index': 105}


In [15]:
g.nodes(data='index')

NodeDataView({'0': 0, '1': 7, '2': 15, '3': 22, '4': 30, '5': 37, '6': 45, '7': 52, '8': 60, '9': 67, '10': 75, '11': 82, '12': 90, '13': 97, '14': 105}, data='index')

In [4]:
cycles = sorted(nx.simple_cycles(g))

In [5]:
dg = nx.DiGraph()

In [6]:
dg